In [17]:
import os
import pandas as pd
import requests
from datetime import datetime

os.chdir("/home/tema/10X/Week8/Logistic optimization/data")
os.getcwd()


'/home/tema/10X/Week8/Logistic optimization/data'

In [25]:
drivers_location = pd.read_csv('drivers_location.csv')
completed_orders = pd.read_csv('completed_orders.csv')
merged_df = pd.read_csv('merged.csv')

In [26]:
completed_orders.drop(columns=['Trip Origin', 'Trip Destination'], inplace = True )
completed_orders.head()

,Trip ID,Trip Start Time,Trip End Time,Trip Duration,Origin Latitude,Origin Longitude,Destination Latitude,Destination Longitude
0,391996,2021-07-01 07:28:04,2021-07-01 07:29:37,1.550000,6.508813,3.377403,6.650970,3.345031
1,391997,2021-07-01 06:38:04,2021-07-01 07:07:28,29.400000,6.431671,3.455538,6.428081,3.472189
2,391998,2021-07-01 06:21:02,2021-07-01 07:02:23,41.350000,6.631679,3.338898,6.508324,3.359040
3,391999,2021-07-01 07:16:07,2021-07-01 07:29:42,13.583333,6.572757,3.367708,6.584881,3.361407
4,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,6.601042,3.276634,6.450107,3.391615


In [28]:
# Convert trip start and end times to datetime
completed_orders['Trip Start Time'] = pd.to_datetime(completed_orders['Trip Start Time'])
completed_orders['Trip End Time'] = pd.to_datetime(completed_orders['Trip End Time'])


In [31]:

# Create new columns for day of the week and hour of the day
completed_orders['Day of Week'] = completed_orders['Trip Start Time'].dt.dayofweek
completed_orders['Hour of Day'] = completed_orders['Trip Start Time'].dt.hour

completed_orders.rename(columns={'Trip Duration': 'Trip Duration(minutes)'}, inplace=True)


In [33]:
# Display a sample of the new features
new_features_sample = completed_orders[['Trip ID', 'Trip Start Time', 'Trip End Time', 'Trip Duration(minutes)', 'Day of Week', 'Hour of Day']].head()

new_features_sample

,Trip ID,Trip Start Time,Trip End Time,Trip Duration(minutes),Day of Week,Hour of Day
0,391996,2021-07-01 07:28:04,2021-07-01 07:29:37,1.550000,3,7
1,391997,2021-07-01 06:38:04,2021-07-01 07:07:28,29.400000,3,6
2,391998,2021-07-01 06:21:02,2021-07-01 07:02:23,41.350000,3,6
3,391999,2021-07-01 07:16:07,2021-07-01 07:29:42,13.583333,3,7
4,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,3,9


In [34]:
completed_orders.dtypes

Trip ID                            int64
Trip Start Time           datetime64[ns]
Trip End Time             datetime64[ns]
Trip Duration(minutes)           float64
Origin Latitude                  float64
Origin Longitude                 float64
Destination Latitude             float64
Destination Longitude            float64
Day of Week                        int32
Hour of Day                        int32
dtype: object

In [ ]:
import pandas as pd
from geopy.distance import geodesic
import haversine as hs
from datetime import datetime

# Load the data
merged_df = pd.read_csv('merged.csv')

In [ ]:
# Helper function to compute geodesic distance
def compute_geodesic_distance(coord1, coord2):
    return geodesic(coord1, coord2).meters

# Helper function to compute haversine distance
def compute_haversine_distance(coord1, coord2):
    return hs.haversine(coord1, coord2) * 1000  # Convert to meters

In [ ]:
# Helper function to compute speed
def compute_speed(distance, start_time, end_time):
    time_diff = (end_time - start_time).total_seconds() / 3600.0  # in hours
    if time_diff == 0:
        return 0
    return distance / time_diff

In [ ]:

merged_df['Trip Start Time'] = pd.to_datetime(merged_df['Trip Start Time'])
merged_df['Trip End Time'] = pd.to_datetime(merged_df['Trip End Time'])


In [35]:

merged_df['Geodesic Distance (m)'] = merged_df.apply(
    lambda row: compute_geodesic_distance((row['Origin Latitude'], row['Origin Longitude']),
                                          (row['Destination Latitude'], row['Destination Longitude'])), axis=1)

merged_df['Haversine Distance (m)'] = merged_df.apply(
    lambda row: compute_haversine_distance((row['Origin Latitude'], row['Origin Longitude']),
                                          (row['Destination Latitude'], row['Destination Longitude'])), axis=1)

merged_df['Speed (m/s)'] = merged_df.apply(
    lambda row: compute_speed(row['Geodesic Distance (m)'], row['Trip Start Time'], row['Trip End Time']), axis=1)

print(merged_df.head())

   id  order_id  driver_id driver_action       lat       lng  Trip ID  \
0   1    392001     243828      accepted  6.602207  3.270465   392001   
1   2    392001     243588      rejected  6.592097  3.287445   392001   
2   3    392001     243830      rejected  6.596133  3.281784   392001   
3   4    392001     243539      rejected  6.596142  3.280526   392001   
4   5    392001     171653      rejected  6.609232  3.288800   392001   

      Trip Start Time       Trip End Time  Trip Duration  Origin Latitude  \
0 2021-07-01 09:30:59 2021-07-01 09:34:36       3.616667         6.601042   
1 2021-07-01 09:30:59 2021-07-01 09:34:36       3.616667         6.601042   
2 2021-07-01 09:30:59 2021-07-01 09:34:36       3.616667         6.601042   
3 2021-07-01 09:30:59 2021-07-01 09:34:36       3.616667         6.601042   
4 2021-07-01 09:30:59 2021-07-01 09:34:36       3.616667         6.601042   

   Origin Longitude  Destination Latitude  Destination Longitude  \
0          3.276634           

In [36]:
merged_df.head()

,id,order_id,driver_id,driver_action,lat,lng,Trip ID,Trip Start Time,Trip End Time,Trip Duration,Origin Latitude,Origin Longitude,Destination Latitude,Destination Longitude,Geodesic Distance (m),Haversine Distance (m),Speed (m/s)
0,1,392001,243828,accepted,6.602207,3.270465,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,6.601042,3.276634,6.450107,3.391615,20984.319062,21048.291126,348126.952179
1,2,392001,243588,rejected,6.592097,3.287445,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,6.601042,3.276634,6.450107,3.391615,20984.319062,21048.291126,348126.952179
2,3,392001,243830,rejected,6.596133,3.281784,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,6.601042,3.276634,6.450107,3.391615,20984.319062,21048.291126,348126.952179
3,4,392001,243539,rejected,6.596142,3.280526,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,6.601042,3.276634,6.450107,3.391615,20984.319062,21048.291126,348126.952179
4,5,392001,171653,rejected,6.609232,3.288800,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,6.601042,3.276634,6.450107,3.391615,20984.319062,21048.291126,348126.952179


In [38]:
merged_df.columns

Index(['id', 'order_id', 'driver_id', 'driver_action', 'lat', 'lng', 'Trip ID',
       'Trip Start Time', 'Trip End Time', 'Trip Duration', 'Origin Latitude',
       'Origin Longitude', 'Destination Latitude', 'Destination Longitude',
       'Geodesic Distance (m)', 'Haversine Distance (m)', 'Speed (m/s)'],
      dtype='object')

In [41]:
# Create new columns for day of the week and hour of the day
merged_df['Day of Week'] = merged_df['Trip Start Time'].dt.dayofweek
merged_df['Hour of Day'] = merged_df['Trip Start Time'].dt.hour

merged_df.rename(columns={'Trip Duration': 'Trip Duration(minutes)'}, inplace=True)

In [42]:
new_features = merged_df[['Trip ID', 'Trip Duration(minutes)', 'Day of Week', 'Hour of Day', 'Geodesic Distance (m)', 'Haversine Distance (m)', 'Speed (m/s)']]

new_features.head()

,Trip ID,Trip Duration(minutes),Day of Week,Hour of Day,Geodesic Distance (m),Haversine Distance (m),Speed (m/s)
0,392001,3.616667,3,9,20984.319062,21048.291126,348126.952179
1,392001,3.616667,3,9,20984.319062,21048.291126,348126.952179
2,392001,3.616667,3,9,20984.319062,21048.291126,348126.952179
3,392001,3.616667,3,9,20984.319062,21048.291126,348126.952179
4,392001,3.616667,3,9,20984.319062,21048.291126,348126.952179


In [43]:
from sklearn.cluster import KMeans
import numpy as np

# Extracting starting and destination coordinates
starting_coords = merged_df[['Origin Latitude', 'Origin Longitude']].drop_duplicates().to_numpy()
destination_coords = merged_df[['Destination Latitude', 'Destination Longitude']].drop_duplicates().to_numpy()

# Clustering starting locations
kmeans_start = KMeans(n_clusters=10, random_state=0).fit(starting_coords)
merged_df['Start Cluster'] = kmeans_start.predict(merged_df[['Origin Latitude', 'Origin Longitude']])

# Clustering destination locations
kmeans_dest = KMeans(n_clusters=10, random_state=0).fit(destination_coords)
merged_df['Destination Cluster'] = kmeans_dest.predict(merged_df[['Destination Latitude', 'Destination Longitude']])

print(merged_df[['order_id', 'Start Cluster', 'Destination Cluster']].head())

/home/tema/10X/Week8/Logistic optimization/Lenv/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
/home/tema/10X/Week8/Logistic optimization/Lenv/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


   order_id  Start Cluster  Destination Cluster
0    392001              7                    9
1    392001              7                    9
2    392001              7                    9
3    392001              7                    9
4    392001              7                    9
